Combining different models to obtain the best performance

In [ ]:
import pandas as pd
import csv
from collections import Counter

# Charger les prédictions de chaque modèle

# tfidf = pd.read_csv('predictions/predictions_tf_lg_7432.csv')
# tfidf2 = pd.read_csv('predictions/predictions_tf_sgc_73898.csv')

fasttext = pd.read_csv('predictions/predictions_fasttext_7946.csv')

mdeberta = pd.read_csv('predictions/predictions_mdebertav3base_86293.csv')

rembert   = pd.read_csv('predictions/predictions_rembert_87445.csv')
rembert2 = pd.read_csv('predictions/predictions_rembert_8683.csv')

xlmroberta = pd.read_csv('predictions/predictions_xlmlast2_87101.csv')
# xlmroberta = pd.read_csv('predictions/predictions_xlmroberta_8647.csv')

bertmulti  = pd.read_csv('predictions/predictions_bertmulti_8656.csv')

bert = pd.read_csv('predictions/predictions_bert_8267.csv')

# mt5 = pd.read_csv('predictions/predictions_mt5_7931.csv')



# Créer un DataFrame combiné en supposant que l'ordre des lignes est identique dans tous les fichiers
df_ensemble = pd.DataFrame()
df_ensemble['ID'] = rembert['ID']  # ou celui de n'importe lequel, si identiques
df_ensemble['mdeberta'] = mdeberta['Label']
df_ensemble['rembert'] = rembert['Label']
df_ensemble['xlmroberta'] = xlmroberta['Label']
df_ensemble['bertmulti'] = bertmulti['Label']
df_ensemble['bert'] = bert['Label']
df_ensemble['fasttext'] = fasttext['Label']
df_ensemble['rembert2'] = rembert2['Label']
# df_ensemble['mt5'] = mt5['Label']
# df_ensemble['tfidf'] = tfidf['Label']

In [ ]:
from collections import Counter

def vote(row):
    # Récupérer la liste des prédictions pour la ligne
    votes = [row['mdeberta'], row['rembert'], row['xlmroberta'], row['bertmulti'], row['bert'], row['rembert2'], row['fasttext']]
    compteur = Counter(votes)
    max_votes = max(compteur.values())
    # Trouver les labels ayant le nombre maximum de votes
    labels_max = [label for label, count in compteur.items() if count == max_votes]
    
    # Si un label a un vote majoritaire unique, le retourner
    if len(labels_max) == 1:
        return labels_max[0]
    else:
        # En cas d'égalité, prioriser RemBERT s'il figure parmi les candidats
        if row['rembert'] in labels_max:
            return row['rembert']
        elif row['xlmroberta'] in labels_max:
            return row['xlmroberta']
        elif row['rembert2'] in labels_max:
            return row['rembert2']
        elif row['bertmulti'] in labels_max:
            return row['bertmulti']
        elif row['mdeberta'] in labels_max:
            return row['mdeberta']
        else:
            # Si aucun des deux n'est dans l'égalité, retourner le premier label de la liste
            return labels_max[0]

# pour prioriser rembert puis xlmroberta puis rembert2 puis bertmulti puis mdeberta

In [ ]:

# Appliquer la fonction de vote sur chaque ligne
df_ensemble['Final_Label'] = df_ensemble.apply(vote, axis=1)

# Créer le fichier de soumission avec les colonnes ID et Label (renommé Final_Label)
submission = df_ensemble[['ID', 'Final_Label']].rename(columns={'Final_Label': 'Label'})

# # Sauvegarder la soumission en CSV
submission.to_csv('predictions/final_submission.csv', index=False)

# Afficher quelques lignes pour vérification
submission.head()


,ID,Label
0,1,ven
1,2,epo
2,3,urd
3,4,que
4,5,chk
